In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


# Data Load

In [2]:
df = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')

In [3]:
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


# Pre-processing

In [4]:
df.isnull().sum()

id                0
model_a           0
model_b           0
prompt            0
response_a        0
response_b        0
winner_model_a    0
winner_model_b    0
winner_tie        0
dtype: int64

In [5]:
#target col
# 0 = A wins, 1 = B wins, 2 = tie
def target_col(row):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1
    elif row['winner_tie'] == 1:
        return 2

df['label'] = df.apply(target_col, axis=1)

In [6]:
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1


## Text encoding using sentence transformer (all-MiniLM-L6-v2)

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

2025-08-06 12:41:07.679144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754484067.880111      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754484067.945642      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Encode prompt, respnse_a, response_b columns
emb_a = model.encode(df['response_a'].tolist())

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

In [9]:
emb_b = model.encode(df['response_b'].tolist())

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

In [10]:
prompt = model.encode(df['prompt'].tolist())

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

# Method 1: sentence transformer -> cosine similarity -> ML 

## Sample Demonstration 

In [11]:
#consider 3rd row
df['response_a'].iloc[2]

'["Function calling is the process of invoking or executing a function in a programming language. To call a function, you need to use the function\'s name followed by parentheses \\"()\\". Within the parentheses, you can pass arguments if the function requires any.\\n\\nHere is an example of how to call a function in Python:\\n\\n```python\\n# Define a function\\ndef greet(name):\\n    print(\\"Hello, \\" + name + \\"!\\")\\n\\n# Call the function\\ngreet(\\"John\\")\\n```\\n\\nIn this example, the function \\"greet\\" is defined with a single parameter \\"name\\" which is used to print a greeting message. To call the function and pass an argument, we write \\"greet(\\"John\\")\\". This will execute the function and print \\"Hello, John!\\".\\n\\nIt is important to note that the function should be defined before calling it in most programming languages, as the interpreter needs to know about the function\'s existence before it can be called."]'

In [12]:
df['response_b'].iloc[2]

'["Function calling is the process of invoking a function in a program. A function is a block of code that performs a specific task and can be reused multiple times in a program. Functions help to organize and modularize code, making it more readable and maintainable.\\n\\nTo call a function, you simply write the name of the function followed by parentheses () containing any arguments or parameters that the function requires. Here\'s an example:\\n```\\nfunction greet(name) {\\n  console.log(\\"Hello, \\" + name);\\n}\\n\\n\\/\\/ Call the greet function and pass in the argument \\"Alice\\"\\ngreet(\\"Alice\\");\\n```\\nIn the example above, the function `greet` takes one parameter, `name`, which is used inside the function to print a greeting message. To call the function, we write the name of the function (`greet`) followed by parentheses containing the argument `\\"Alice\\"`. When the function is called, the argument `\\"Alice\\"` is passed to the function, where it is assigned to th

In [13]:
a = """Function calling is the process of invoking or executing a function in a programming language. To call a function, you need to use the function\'s name followed by parentheses \\"()\\". Within the parentheses, you can pass arguments if the function requires any.\\n\\nHere is an example of how to call a function in Python:\\n\\n```python\\n# Define a function\\ndef greet(name):\\n    print(\\"Hello, \\" + name + \\"!\\")\\n\\n# Call the function\\ngreet(\\"John\\")\\n```\\n\\nIn this example, the function \\"greet\\" is defined with a single parameter \\"name\\" which is used to print a greeting message. To call the function and pass an argument, we write \\"greet(\\"John\\")\\". This will execute the function and print \\"Hello, John!\\".\\n\\nIt is important to note that the function should be defined before calling it in most programming languages, as the interpreter needs to know about the function\'s existence before it can be called."""
b = """Function calling is the process of invoking a function in a program. A function is a block of code that performs a specific task and can be reused multiple times in a program. Functions help to organize and modularize code, making it more readable and maintainable.\\n\\nTo call a function, you simply write the name of the function followed by parentheses () containing any arguments or parameters that the function requires. Here\'s an example:\\n```\\nfunction greet(name) {\\n  console.log(\\"Hello, \\" + name);\\n}\\n\\n\\/\\/ Call the greet function and pass in the argument \\"Alice\\"\\ngreet(\\"Alice\\");\\n```\\nIn the example above, the function `greet` takes one parameter, `name`, which is used inside the function to print a greeting message. To call the function, we write the name of the function (`greet`) followed by parentheses containing the argument `\\"Alice\\"`. When the function is called, the argument `\\"Alice\\"` is passed to the function, where it is assigned to the `name` parameter. The function then uses the `name` parameter to print the greeting message `\\"Hello, Alice\\"`.\\n\\nFunctions can also return values, which can be assigned to variables or used in expressions. For example:\\n```\\nfunction addNumbers(a, b) {\\n  return a + b;\\n}\\n\\n\\/\\/ Call the addNumbers function and assign the result to the variable sum\\nvar sum = addNumbers(3, 5);\\nconsole.log(sum); \\/\\/ Output: 8\\n```\\nIn the example above, the function `addNumbers` takes two parameters, `a` and `b`, adds them together, and returns the result. To call the function, we write the name of the function (`addNumbers`) followed by parentheses containing the arguments `3` and `5`. The function calculates the sum of the two arguments (`3 + 5 = 8`) and returns the result (`8`). We then assign the result to the variable `sum` and print it to the console."""

In [14]:
encode_a = model.encode([a])[0]
encode_b = model.encode([b])[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
from scipy.spatial import distance
similarity_score = 1 - distance.cosine(encode_b, encode_a)
print(f'similarity score of a and b is {similarity_score}')

similarity score of a and b is 0.8515121252245993


In [16]:
df['prompt'].iloc[2]

'["explain function calling. how would you call a function?"]'

In [17]:
p = """explain function calling. how would you call a function?"""
encode_p = model.encode([p])[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
ss = 1 - distance.cosine(encode_p, encode_a)
s3 = 1 - distance.cosine(encode_p, encode_b)
print(ss, s3)

0.6581653356552124 0.7132453151491485


In [19]:
a1 = model.encode([df['prompt'].iloc[0]])[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
p1 = model.encode([df['response_a'].iloc[0]])[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
s4 = 1 - distance.cosine(a1, p1)
s4

0.4095202088356018

## Model

In [22]:
from scipy.spatial.distance import cosine
similarities_prompt_a = []
similarities_prompt_b = []
similarities_a_b = []

for p, a, b in zip(prompt, emb_a, emb_b):
    sim_pa = 1 - cosine(p, a)
    sim_pb = 1 - cosine(p, b)
    sim_ab = 1 - cosine(a, b)

    similarities_prompt_a.append(sim_pa)
    similarities_prompt_b.append(sim_pb)
    similarities_a_b.append(sim_ab)

# Store results in DataFrame
df['sim_prompt_a'] = similarities_prompt_a
df['sim_prompt_b'] = similarities_prompt_b
df['sim_a_b'] = similarities_a_b

In [23]:
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,sim_prompt_a,sim_prompt_b,sim_a_b
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0,0.409520,0.560817,0.537583
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,1,0.710832,0.618878,0.700466
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,2,0.568760,0.611341,0.811002
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,0,0.631789,0.550531,0.798137
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1,0.636248,0.691548,0.910706


### Splitting the data 

In [24]:
from sklearn.model_selection import train_test_split
X = df[['sim_prompt_a', 'sim_prompt_b', 'sim_a_b']]
X_train, X_test, y_train, y_test = train_test_split(X, df['label'],test_size=0.2)

In [25]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state = 42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [26]:
y_probs_rf = clf.predict_proba(X_test)
from sklearn.metrics import log_loss
print("Log Loss:", log_loss(y_test, y_probs_rf))

Log Loss: 1.1794865777875745


In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [28]:
y_lr_probs = lr.predict_proba(X_test)
print(log_loss(y_test, y_lr_probs))

1.0942708297767847


In [29]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(num_leaves=31, max_depth=-1, learning_rate=0.05, n_estimators=100)

lgb_model.fit(X_train, y_train)
y_lgb_probs = lgb_model.predict_proba(X_test)
print(log_loss(y_test, y_lgb_probs))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001682 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 45981, number of used features: 3
[LightGBM] [Info] Start training from score -1.053143
[LightGBM] [Info] Start training from score -1.075141
[LightGBM] [Info] Start training from score -1.171471
1.0863695590305733


# Method 2: sentence transformer -> differences -> ML

In [30]:
features = np.hstack([
    np.abs(emb_a - emb_b),
    np.abs(prompt - emb_a),
    np.abs(prompt - emb_b)
])

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X2 = features
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, df['label'], test_size=0.2)

In [32]:
clf2 = RandomForestClassifier(n_estimators=100, random_state = 42)
clf2.fit(X2_train, y2_train)

RandomForestClassifier(random_state=42)

In [33]:
y2_probs = clf2.predict_proba(X2_test)
from sklearn.metrics import log_loss
print("Log Loss:", log_loss(y2_test, y2_probs))

Log Loss: 1.0942716416273817


In [34]:
import lightgbm as lgb
lgb_model2 = lgb.LGBMClassifier(num_leaves=31, max_depth=-1, learning_rate=0.05, n_estimators=100)

# Train the model
lgb_model2.fit(X2_train, y2_train)
y_lgb_probs = lgb_model2.predict_proba(X2_test)
print(log_loss(y2_test, y_lgb_probs))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.753880 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 293760
[LightGBM] [Info] Number of data points in the train set: 45981, number of used features: 1152
[LightGBM] [Info] Start training from score -1.048912
[LightGBM] [Info] Start training from score -1.071642
[LightGBM] [Info] Start training from score -1.180140
1.077635504793599


### best model: difference funstion with  lgbm clssifier

# Test Data

In [35]:
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

In [36]:
test.head()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [37]:
test.isnull().sum()

id            0
prompt        0
response_a    0
response_b    0
dtype: int64

In [38]:
test_emb_a = model.encode(test['response_a'].tolist())
test_emb_b = model.encode(test['response_b'].tolist())
test_prompt = model.encode(test['prompt'].tolist())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
#cosine similarity
from scipy.spatial.distance import cosine
test_similarities_prompt_a = []
test_similarities_prompt_b = []
test_similarities_a_b = []

for p, a, b in zip(test_prompt, test_emb_a, test_emb_b):
    t_sim_pa = 1 - cosine(p, a)
    t_sim_pb = 1 - cosine(p, b)
    t_sim_ab = 1 - cosine(a, b)

    test_similarities_prompt_a.append(t_sim_pa)
    test_similarities_prompt_b.append(t_sim_pb)
    test_similarities_a_b.append(t_sim_ab)

# Store results in DataFrame
test['sim_prompt_a'] = test_similarities_prompt_a
test['sim_prompt_b'] = test_similarities_prompt_b
test['sim_a_b'] = test_similarities_a_b

In [40]:
test.head()

,id,prompt,response_a,response_b,sim_prompt_a,sim_prompt_b,sim_a_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran...",0.824369,0.856802,0.856265
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ...",0.378464,0.653580,0.362643
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p...",0.758472,0.738682,0.836689


In [41]:
#difference function
test_features = np.hstack([
    np.abs(test_emb_a - test_emb_b),
    np.abs(test_prompt - test_emb_a),
    np.abs(test_prompt - test_emb_b)
])

In [42]:
# X3 = test[['sim_prompt_a', 'sim_prompt_b', 'sim_a_b']]
test_probs = lgb_model2.predict_proba(test_features)

In [43]:
test_probs

array([[0.25966528, 0.31263647, 0.42769825],
       [0.31265949, 0.3722979 , 0.31504261],
       [0.35611913, 0.26900901, 0.37487186]])

In [44]:
submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': test_probs[:, 0],
    'winner_model_b': test_probs[:, 1],
    'winner_tie': test_probs[:, 2],
})

In [45]:
submission.to_csv('submission.csv', index=False)